In [5]:
import gc
import re
from collections import defaultdict
from pprint import pprint

import nltk
import pandas as pd
import torch
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from pymystem3 import Mystem
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder, SentenceTransformer, util
from tqdm import tqdm

In [6]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Предобработка текстов

In [7]:
morph_analyzer = Mystem()
tokenizer = WordPunctTokenizer()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [8]:
russian_stopwords = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"

russian_stopwords.extend(additional_sw.split())
russian_stopwords.extend(pronouns.split())
russian_stopwords.extend(conjunctions.split())
russian_stopwords.extend(digits.split())
russian_stopwords.extend(modal_words.split())
russian_stopwords.extend(particles.split())
russian_stopwords.extend(prepositions.split())
russian_stopwords.extend(evaluative.split())
russian_stopwords = set(russian_stopwords)

In [9]:
def lemmatize(words) -> list[str]:
    lemmatized_text = morph_analyzer.lemmatize(" ".join(words))
    return [word for word in lemmatized_text if word.isalnum()]


def delete_stop_words(words):
    return [word for word in words if word not in russian_stopwords]


def delete_punctuation(text):
    return " ".join([word for word in tokenizer.tokenize(text) if word.isalnum()])


def tokenize_by_sentences(text: str) -> list[str]:
    return sent_tokenize(text, language="russian")


def tokenize_by_words(text):
    return word_tokenize(text, language="russian")


def convert_text_to_lowercase(text: str):
    return text.lower()


def preprocess(text: str) -> str:
    sentences: list[str] = tokenize_by_sentences(text)
    preprocessed_sentences: list[str] = list()
    for sentence in sentences:
        sentence = convert_text_to_lowercase(sentence)
        sentence = delete_punctuation(sentence)
        words = tokenize_by_words(sentence)
        words = delete_stop_words(words)
        words = lemmatize(words)
        preprocessed_sentences.extend(words)
    return " ".join(preprocessed_sentences)

## Датасет

In [10]:
df = pd.read_csv(
    "/content/drive/MyDrive/resume_job_matching/Raw_Jobs.csv", delimiter=";"
)
df.head()

,№,id,title,salary,experience,job_type,description,key_skills,company,location,date_of_post,type
0,1,78577559,Главный механик,от 160000 до 160000 RUR,Более 6 лет,"Полная занятость,полный день","Вакансия компании: ООО ПК Предприятие ""ПИК"" ...",NaN,JCat.ru,Владивосток,27.03.2023,close
1,2,78693069,HTML-верстальщик,з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",В креативное агентство на полный рабочий день ...,NaN,Чугунова Ирина,NaN,29.03.2023,close
2,3,78945452,Тренер (направление профессионального обучения),з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",Наши предложения: самостоятельность в...,NaN,Перекрёсток,NaN,18.04.2023,close
3,4,79352008,Старший механик на КСПГ,з/п не указана,Более 6 лет,"Полная занятость,полный день",Обязанности: _Обеспечение технически ...,NaN,Газпром гелий сервис,NaN,17.04.2023,close
4,5,79406472,System Analyst Trainee,з/п не указана,Нет опыта,"Стажировка,полный день",IT-компания Aston - компания по разработке про...,"UML,SQL,Scrum,Retail,Базы данных,Kanban,Waterf...",Aston,NaN,17.04.2023,close


In [11]:
valid_comment_pattern = r"программист|разработчик|developer"

In [12]:
df["is_valid_pattern"] = df["title"].apply(
    lambda x: True
    if not isinstance(x, float)
    and re.match(valid_comment_pattern, x.lower()) is not None
    else False
)

In [13]:
df = df[df.is_valid_pattern == True]

In [14]:
def extract_information(job_description):
    responsibilities_pattern = r"\b(?:вам предстоит|вы будете|задачи|обязанности|чем необходимо заниматься)\b:(.*?)(?=\b(?:требования|требования к кандидатам|мы предлагаем|условия|требования:|мы ожидаем|наши пожелания|будет плюсом|будет преимуществом)\b|$)"
    requirements_pattern = r"\b(?:для нас важно|ждем от вас|мы ожидаем|наши пожелания|необходимые навыки|требования|требования к кандидатам|будет плюсом|будет преимуществом)\b:(.*?)(?=\b(?:мы предлагаем|условия|место работы|длительность рабочего дня)\b|$)"
    conditions_pattern = r"\b(?:мы предлагаем|условия)\b:(.*?)(?=\b(?:в ответ на оставленный отклик|образование)\b|$)"

    responsibilities_match = re.search(
        responsibilities_pattern, job_description, re.IGNORECASE | re.DOTALL
    )
    requirements_match = re.search(
        requirements_pattern, job_description, re.IGNORECASE | re.DOTALL
    )
    conditions_match = re.search(
        conditions_pattern, job_description, re.IGNORECASE | re.DOTALL
    )

    responsibilities = (
        responsibilities_match.group(1).strip() if responsibilities_match else None
    )
    requirements = requirements_match.group(1).strip() if requirements_match else None
    conditions = conditions_match.group(1).strip() if conditions_match else None

    return responsibilities, requirements, conditions

In [15]:
responsabilities, req, cond = list(), list(), list()

In [16]:
for row in tqdm(df["description"]):
    responsability, requirements, conditions = extract_information(row)
    responsabilities.append(responsability)
    req.append(requirements)
    cond.append(conditions)

100%|██████████| 1219/1219 [00:00<00:00, 1584.72it/s]


In [17]:
df["responsabilities"] = responsabilities
df["requirements"] = req
df["conditions"] = cond

In [18]:
del responsabilities
del req
del cond
gc.collect()

60

In [19]:
vacancy_text = defaultdict(str)
for idx, row in df.iterrows():
    vacancy_text[
        idx
    ] = f"Профессия: {row['title']}. Ключевые навыки: {row['key_skills']}. Обязанности: {row['responsabilities']}. Требования: {row['requirements']}. Условия {row['conditions']}"

In [20]:
df["text"] = df.index.map(vacancy_text)

In [21]:
query = """Программист C++, Web-программист, разработчик аппаратуры,.Опыт работы 5 лет 8 месяцев  Март 2019 — по настоящее время 3 месяца Удаленная работа Обнинск Web-программист Доработка сайта www.autolarek24.ru  Октябрь 2013 — по настоящее время 5 лет 8 месяцев АО «ГНЦ РФ – ФЭИ» Обнинск , www.ippe.ru Энергетика ... Атомная энергетика (генерация электроэнергии, АЭС) Инженер-программист Разработка ПО для операционных систем жесткого реального времени. Разработка драйверов, многопоточных приложений, клиент-серверных приложений. Разработка микропроцессорных систем. Работа с промышленными интерфейсами и протоколами: ModBus, CAN, RS422/485. Разработка документации ПО. Работа с измерительной аппаратурой с очень высокой точностью измерения и написание ПО для мониторинга и фильтрации измерений, а так же участие в разработке, настройке и тестировании данной аппаратуры.  Июнь 2017 — Ноябрь  2017 6 месяцев ООО "Персона" Обнинск Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Веб-разработчик Разработка веб-сайтов, концепция MVC, командная разработка (Git), Laravel. Проект www.combeep.com"""

# Алгоритм работы поиска вакансий по резюме

Данный алгоритм состоит из трех шагов:

1. Сначала отбираем топ релевантных вакансий с помощью BM25.
2. Затем сужаем нашу воронку полученных вакансий через multilingual-e5-large
3. Реранжируем кросс-энкодером

## Ищем топ релевантных вакансий с помощью ВM25

In [46]:
docs = df["text"].tolist()
preprocessed_docs = [doc for doc in docs]

In [47]:
tokenized_docs = [doc.split() for doc in preprocessed_docs]
bm25 = BM25Okapi(tokenized_docs)

In [48]:
def bm25_search(query):
    scores = bm25.get_scores(query.split())
    return scores

In [49]:
bm25_scores = bm25_search(query)

In [50]:
bm25_scores

array([32.58946245, 36.53600234, 50.07421609, ...,  1.05817918,
       39.72927907, 31.37882932])

## Сужаем воронку

In [51]:
model = SentenceTransformer("intfloat/multilingual-e5-large")

In [52]:
def encoder_narrowing(query, candidates):
    query_embedding = model.encode(query)
    candidate_embeddings = model.encode(candidates)
    similarities = util.pytorch_cos_sim(query_embedding, candidate_embeddings)[0]
    sorted_indices = similarities.argsort().tolist()[::-1]
    narrowed_candidates = [candidates[i] for i in sorted_indices]
    return narrowed_candidates[:10]  # Выбираем топ 5 кандидатов

In [53]:
top_candidates = [doc for _, doc in sorted(zip(bm25_scores, docs), reverse=True)[:10]]

In [54]:
narrowed_candidates = encoder_narrowing(query, top_candidates)

In [55]:
narrowed_candidates

['Профессия: Разработчик Python (junior). Ключевые навыки: Python,Git,Mercurial,ООП,REST,Angular. Обязанности: разработка нового и поддержка существующего функционала ИС ЛОД.. Требования: — Опыт работы разработчиком ПО не менее 1 года.   — Опыт разработки на Python.   — Опыт использования реляционных баз данных.   — Свободное владение git / Mercurial.   — Знание и понимание основных принципов ООП.   — Знание и понимание основных принципов REST.         Дополнительными плюсами будут:     — Angular (готовность изучить для решения задач по фронту).   — Опыт разработки web-приложений на python с использованием одного из следующих фреймворков: Django, Flask, FastApi и др.   — Опыт работы в Redmine / Jira.   — Опыт использования ORM.   — Опыт работы в Linux.. Условия — Возможность работать в технологической компании, создающей сервисы и системы полезные людям.   — Официальное оформление с первого дня работы и все соц.гарантии по ТК РФ   — Аккредитованная IT-компания: льготы для сотрудников  

## Реранкер

In [56]:
cross_model = CrossEncoder("PitKoro/cross-encoder-ru-msmarco-passage", max_length=2048)

In [57]:
def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores

In [58]:
model_inputs = [[query, item[:1024]] for item in narrowed_candidates]
scores = cross_score(model_inputs)
# Sort the scores in decreasing order
ranked_results = [
    {"Text": inp, "Score": score} for inp, score in zip(narrowed_candidates, scores)
]
ranked_results = sorted(ranked_results, key=lambda x: x["Score"], reverse=True)


print("\n")
for result in ranked_results:
    print("\t", pprint(result))



{'Score': 0.52775455,
 'Text': 'Профессия: Программист. Ключевые навыки: '
         'C/C++,Linux,ARM,Планирование карьеры,C++. Обязанности: Разработка '
         'алгоритмов и написание программ для процессоров и контроллеров '
         'электромеханических и робототехнических систем в соответствии с '
         'техническим заданием     Тестирование и отработка систем и '
         'программного обеспечения     Участие в проверке, настройке и наладке '
         'систем и программного обеспечения совместно с разработчиками '
         'аппаратной платформы     Разработка и оформление документации на '
         'программное обеспечение по требованию заказчика в соответствии с '
         'нормативами     Участие в разработке текстовой документации на '
         'систему. Требования: Знание языка С как преимущество (либо С++, '
         'С#)     Опыт программирования контроллеров и процессоров '
         'встраиваемых систем ARM,TMS,STM 32, Cortex etc.     Умение работать '
         'с циф

## Метрики

Посчитаем метрики на размеченном датасете для IT вакансий

In [60]:
labeling = pd.read_csv("/content/drive/MyDrive/resume_job_matching/labeling.csv")

In [61]:
top100_vacancies = pd.read_csv('/content/drive/MyDrive/resume_job_matching/random100vac.csv')
top100_vacancies.head()

,№,id,Должность,Зарплата,Опыт работы,Занятость,Описание вакансии,Ключевые навыки,Компания,Город,Обновление резюме,Тип,is_valid_pattern,text
0,394,79011638,Разработчик Kotlin,з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день","Marlerino Group – молодая, развивающаяся I...","Java,Мобильные приложения,ООП,Kotlin,Android S...",Marlerino Group,Россия,06.04.2023,close,True,Разработчик Kotlin. Marlerino Group – моло...
1,426,79280145,Разработчик Bitrix24,з/п не указана,От 3 до 6 лет,"Полная занятость,полный день",Обязанности: Техническая поддержка П...,ключевые навыки отсутствуют,Медицинские товары,Россия,13.04.2023,close,True,Разработчик Bitrix24. Обязанности: Т...
2,1233,79472031,Программист 1С (начальный уровень),от 60000 RUR,От 1 года до 3 лет,"Полная занятость,полный день","В компанию, имеющую аккредитацию ИТ компаний, ...","1С: Зарплата и управление персоналом,1С: Бухга...",МАКСИМА,Россия,18.04.2023,close,True,Программист 1С (начальный уровень). В компанию...
3,1429,79600291,Программист станков с ЧПУ,от 35000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",Обязанности: Программирование станков...,ключевые навыки отсутствуют,КНИИТМУ,"Калуга, улица Карла Маркса, 4",20.04.2023,close,True,Программист станков с ЧПУ. Обязанности: ...
4,2336,78562452,Разработчик / аналитик Qlik Sense,от 130000 до 150000 RUR,От 3 до 6 лет,"Полная занятость,полный день",Задачи: • Участие в формировании стратеги...,"Qlik Sense,BI,QlikView,Анализ бизнес показател...",А-Контракт,Россия,27.03.2023,close,True,Разработчик / аналитик Qlik Sense. Задачи: ...


In [62]:
top100_resume = pd.read_csv('/content/drive/MyDrive/resume_job_matching/random100resume.csv')
top100_resume.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,is_valid_pattern,text
0,"Мужчина , 27 лет , родился 8 августа 1991",70000 руб.,"Программист C++, Web-программист, разработчик ...","Обнинск , готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день, сменный график, уд...",Опыт работы 5 лет 8 месяцев Март 2019 — по на...,Удаленная работа,Web-программист,Высшее образование 2018 ИАТЭ НИЯУ МИФИ Институ...,17.04.2019 16:49,Имеется собственный автомобиль,True,"Программист C++, Web-программист, разработчик ..."
1,"Мужчина , 24 года , родился 14 июля 1994",70000 руб.,Программист,"Тюмень , не готов к переезду , не готов к кома...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы 3 года 2 месяца Декабрь 2018 — по...,Индивидуальное предпринимательство / частная п...,Программист,Высшее образование 2017 НГИИ (бывш. НИИ) Факул...,13.05.2019 12:01,Не указано,True,Программист.Опыт работы 3 года 2 месяца Декаб...
2,"Женщина , 44 года , родилась 15 октября 1974",50000 руб.,Программист,"Губкин , готова к переезду , готова к командир...",полная занятость,"полный день, сменный график, удаленная работа",Опыт работы 12 лет 5 месяцев Декабрь 2016 — А...,"ООО""ТД Молочный мир""",Торговый представитель,Высшее образование 2011 Московский Государстве...,12.04.2019 12:54,Не указано,True,Программист.Опыт работы 12 лет 5 месяцев Дека...
3,"Мужчина , 24 года , родился 13 июня 1994",30000 руб.,Программист-разработчик,"Ростов-на-Дону , готов к переезду , готов к ко...",полная занятость,полный день,Опыт работы 2 года 11 месяцев Июль 2016 — по ...,"ООО ""АПС""",Инженер,Высшее образование (Магистр) 2018 Ростовский ...,12.04.2019 11:52,Не указано,True,Программист-разработчик.Опыт работы 2 года 11 ...
4,"Мужчина , 35 лет , родился 12 марта 1984",1800 руб.,Программист 1С 8,"Москва , м. Выхино , не готов к переезду , не...","частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 16 лет 2 месяца Декабрь 2007 — по...,Комплексное сопровождение клиентов по автомати...,Программист 1С,Высшее образование 2008 Южно-Российский госуда...,07.05.2019 11:34,Не указано,True,Программист 1С 8.Опыт работы 16 лет 2 месяца ...


In [63]:
labeling.head()

,Unnamed: 0,is_relevant,score,interpretation,resume_index,vacancy_index
0,0,0,2,"Вакансия не подходит, поскольку опыт работы в ...",83,4
1,1,0,0,"Вакансия не соответствует опыту кандидата, так...",83,63
2,2,0,0,"Вакансия не релевантна, поскольку требуется сп...",83,3
3,3,0,0,"Отсутствие в резюме упоминания о знании PHP, L...",83,36
4,4,0,2,Не подходит: опыт работы соискателя значительн...,83,44


In [64]:
vacancies = top100_vacancies[top100_vacancies.index.isin(labeling.vacancy_index.unique())]
vacancies.head()

,№,id,Должность,Зарплата,Опыт работы,Занятость,Описание вакансии,Ключевые навыки,Компания,Город,Обновление резюме,Тип,is_valid_pattern,text
1,426,79280145,Разработчик Bitrix24,з/п не указана,От 3 до 6 лет,"Полная занятость,полный день",Обязанности: Техническая поддержка П...,ключевые навыки отсутствуют,Медицинские товары,Россия,13.04.2023,close,True,Разработчик Bitrix24. Обязанности: Т...
3,1429,79600291,Программист станков с ЧПУ,от 35000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",Обязанности: Программирование станков...,ключевые навыки отсутствуют,КНИИТМУ,"Калуга, улица Карла Маркса, 4",20.04.2023,close,True,Программист станков с ЧПУ. Обязанности: ...
4,2336,78562452,Разработчик / аналитик Qlik Sense,от 130000 до 150000 RUR,От 3 до 6 лет,"Полная занятость,полный день",Задачи: • Участие в формировании стратеги...,"Qlik Sense,BI,QlikView,Анализ бизнес показател...",А-Контракт,Россия,27.03.2023,close,True,Разработчик / аналитик Qlik Sense. Задачи: ...
5,2350,79071089,Разработчик фронтенда в команду API Карт,з/п не указана,От 3 до 6 лет,"Полная занятость,полный день",Мы разрабатываем JavaScript API Яндекс Карт. Н...,"JavaScript,TypeScript,React,API",Яндекс,"Москва, Садовническая улица, 82с2",09.04.2023,close,True,Разработчик фронтенда в команду API Карт. Мы р...
6,2595,78050064,Программист -тестировщик (отдел автотестирования),от 46000 до 92000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",Тензор - это аккредитованная IT-компания: боле...,ключевые навыки отсутствуют,Тензор,Россия,10.04.2023,close,True,Программист -тестировщик (отдел автотестирован...


In [65]:
resumes  = top100_resume[top100_resume.index.isin(labeling.resume_index.unique())]
resumes.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,is_valid_pattern,text
1,"Мужчина , 24 года , родился 14 июля 1994",70000 руб.,Программист,"Тюмень , не готов к переезду , не готов к кома...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы 3 года 2 месяца Декабрь 2018 — по...,Индивидуальное предпринимательство / частная п...,Программист,Высшее образование 2017 НГИИ (бывш. НИИ) Факул...,13.05.2019 12:01,Не указано,True,Программист.Опыт работы 3 года 2 месяца Декаб...
6,"Мужчина , 22 года , родился 27 октября 1996",100000 руб.,Программист С++,"Москва , м. Войковская , не готов к переезду ...","проектная работа, частичная занятость","удаленная работа, гибкий график",Опыт работы 1 год 2 месяца Программист С++ 10...,Design Dossier,Программист,Неоконченное высшее образование 2020 Московск...,22.04.2019 16:06,Не указано,True,Программист С++.Опыт работы 1 год 2 месяца Пр...
26,"Мужчина , 48 лет , родился 17 октября 1970",30000 руб.,"Программист, оператор ПК","Санкт-Петербург , м. Парк Победы , готов к пе...","проектная работа, стажировка, частичная занято...","удаленная работа, гибкий график, полный день, ...",Опыт работы 23 года 6 месяцев Март 1995 — Авг...,ГК Бюллетень Недвижимости,Программист,Неоконченное высшее образование 1992 Санкт-Пе...,07.05.2019 21:02,Не указано,True,"Программист, оператор ПК.Опыт работы 23 года 6..."
30,"Мужчина , 36 лет , родился 14 февраля 1983",60000 руб.,Программист-разработчик,"Москва , м. Алтуфьево , готов к переезду (Бел...","частичная занятость, проектная работа, полная ...",удаленная работа,Опыт работы 9 лет 10 месяцев Август 2018 — Ма...,"ОАО ""Вента""",Программист,Высшее образование 2005 Уральский федеральный...,17.04.2019 15:13,Не указано,True,Программист-разработчик.Опыт работы 9 лет 10 м...
33,"Мужчина , 31 год , родился 21 октября 1987",50000 руб.,Разработчик ПО,"Набережные Челны , готов к переезду , готов к ...","частичная занятость, проектная работа, полная ...","полный день, сменный график, вахтовый метод, у...",Опыт работы 6 лет 2 месяца Разработчик ПО 50 ...,БАРС Груп,Web-разработчик,Высшее образование (Бакалавр) 2017 Филиал Каз...,22.04.2019 08:59,Не указано,True,Разработчик ПО.Опыт работы 6 лет 2 месяца Раз...


Построим модель на датасете резюме

In [66]:
docs = vacancies["text"].tolist()
preprocessed_docs = [doc for doc in docs]
tokenized_docs = [doc.split() for doc in preprocessed_docs]
bm25 = BM25Okapi(tokenized_docs)

In [67]:
def get_results_by_query(query, top_k: int = 5):
  bm25_scores = bm25_search(query)
  top_candidates = [doc for _, doc in sorted(zip(bm25_scores, docs), reverse=True)[:top_k]]
  narrowed_candidates = encoder_narrowing(query, top_candidates)
  model_inputs = [[query, item[:1024]] for item in narrowed_candidates]
  scores = cross_score(model_inputs)
  # Sort the scores in decreasing order
  ranked_results = [
      {"Text": inp, "Score": score} for inp, score in zip(narrowed_candidates, scores)
  ]
  ranked_results = sorted(ranked_results, key=lambda x: x["Score"], reverse=True)
  return ranked_results


In [73]:
vacancies.reset_index(inplace=True)

In [77]:
result_dict = defaultdict(list)
for idx, row in resumes.iterrows():
  resume_inx = top100_resume.index[top100_resume.text == row['text']].tolist()[0]
  top_k = 10
  ranked_results = get_results_by_query(row['text'][:1024], top_k = top_k)
  ranked_results_internal_vac_idx  = vacancies.index[vacancies.text.isin([vac['Text'] for vac in ranked_results])].tolist()
  internal_vac_texts = vacancies.iloc[ranked_results_internal_vac_idx]['text'].tolist()
  external_vac_indexes = top100_vacancies.index[top100_vacancies.text.isin(internal_vac_texts)].tolist()
  result_dict[resume_inx] = external_vac_indexes

In [78]:
labeling_dict = defaultdict(set)
for idx, row in labeling.iterrows():
  labeling_dict[row['resume_index']].add(row['vacancy_index'])

In [79]:
def precision(groundtruth,predicted):
   return len(set(predicted).intersection(set(groundtruth))) / len(predicted)
def recall(groundruth, predicted):
   return len(set(predicted).intersection(set(groundtruth))) / len(groundtruth)

def f1(precision, recall):
  return (2 * (precision * recall) / (precision + recall)) if (precision + recall) > 0 else 0

In [82]:
precisions, recalls, f1s = list(), list(), list()
for resume_idx, vacancy_idxs in result_dict.items():
  predicted   = vacancy_idxs
  groundtruth = labeling_dict[resume_idx]
  prec = precision(groundtruth, predicted)
  rec = recall(groundtruth, predicted)
  f1s.append(f1(prec, rec))
  precisions.append(prec)
  recalls.append(rec)

print(f"Mean precision: {sum(precisions) / len(precisions)}")
print(f"Mean recall: {sum(recalls) / len(recalls)}")
print(f"Mean F1: {sum(f1s) / len(f1s)}")

Mean precision: 0.15456212
Mean recall: 0.11123456
Mean F1: 0.12936692370173472
